## Thomson Numerical Optimization

In [ ]:
import torch

# Parameters
N = 6  # Number of points
lr = 0.01  # Learning rate
steps = 1000  # Number of optimization steps

# Initialize points randomly on the sphere
points = torch.randn(N, 3, requires_grad=True)
points.data /= points.data.norm(dim=1, keepdim=True)  # Normalize to lie on the sphere

# Optimizer
optimizer = torch.optim.Adam([points], lr=lr)

# Optimization loop
for step in range(steps):
    # Zero the gradients
    optimizer.zero_grad()

    # Compute the energy
    energy = 0
    for i in range(N):
        for j in range(i + 1, N):
            distance = torch.norm(points[i] - points[j])
            energy += 1 / distance  # Coulomb potential

    # Backpropagate to compute gradients
    energy.backward()

    # Project gradients onto the tangent space of the sphere
    with torch.no_grad():
        grad = points.grad
        tangent_grad = grad - (grad * points).sum(dim=1, keepdim=True) * points
        points.grad.copy_(tangent_grad)  # Replace the gradient with the tangent component

    # Take an optimizer step
    optimizer.step()

    # Reproject points back onto the sphere
    with torch.no_grad():
        points.data /= points.data.norm(dim=1, keepdim=True)

    # Print progress
    if step % 100 == 0:
        print(f"Step {step}, Energy: {energy.item()}")

# Final result
print("Optimized Points:")
print(points)
